In [22]:
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import matplotlib
import keras.backend as K
matplotlib.style.use('ggplot')

In [24]:
import os
IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = os.path.join(os.getcwd(), 'training')
VALID_DATA_DIR = os.path.join(os.getcwd(), 'validation')

1440


In [25]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

train_generator = datagen.flow_from_directory(
    TRAINING_DATA_DIR,
    shuffle=True,
    target_size=IMAGE_SHAPE,
)


Found 2628 images belonging to 2 classes.


In [26]:

valid_generator = datagen.flow_from_directory(
    VALID_DATA_DIR,
    shuffle=False,
    target_size=IMAGE_SHAPE,
)

Found 657 images belonging to 2 classes.


In [27]:
def build_model(num_classes):
    model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', 
                           input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model
model = build_model(num_classes=2)

In [28]:
def get_f1(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    
    return f1_val

In [29]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[get_f1]
)
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 111, 111, 16)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 109, 109, 16)      2320      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 54, 54, 16)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 52, 52, 32)        4640      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 26, 26, 32)      

In [34]:
EPOCHS = 8
BATCH_SIZE = 128
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // BATCH_SIZE // 2,
                    epochs=EPOCHS,
                    validation_data=valid_generator,
                    validation_steps= valid_generator.samples // BATCH_SIZE // 2,
                    verbose=1
                    )

Epoch 1/8
10/10 [==============================] - 5s 496ms/step - loss: 0.0462 - get_f1: 0.9875 - val_loss: 0.1430 - val_get_f1: 0.9688
Epoch 2/8
10/10 [==============================] - 5s 447ms/step - loss: 0.1110 - get_f1: 0.9719 - val_loss: 0.1695 - val_get_f1: 0.9375
Epoch 3/8
10/10 [==============================] - 5s 436ms/step - loss: 0.0677 - get_f1: 0.9844 - val_loss: 0.1113 - val_get_f1: 0.9688
Epoch 4/8
10/10 [==============================] - 5s 450ms/step - loss: 0.0483 - get_f1: 0.9906 - val_loss: 0.1625 - val_get_f1: 0.9531
Epoch 5/8
10/10 [==============================] - 5s 445ms/step - loss: 0.1004 - get_f1: 0.9500 - val_loss: 0.1027 - val_get_f1: 0.9688
Epoch 6/8
10/10 [==============================] - 5s 460ms/step - loss: 0.0767 - get_f1: 0.9812 - val_loss: 0.2513 - val_get_f1: 0.9531
Epoch 7/8
10/10 [==============================] - 5s 468ms/step - loss: 0.0726 - get_f1: 0.9750 - val_loss: 0.1590 - val_get_f1: 0.9688
Epoch 8/8
10/10 [========================

In [44]:
if not os.path.isdir('Ayhem_Bouabid'):
    model.save("Ayhem_Bouabid")

In [38]:
from tensorflow import keras
reconstructed_model = keras.models.load_model('Ayhem_Bouabid', compile=False)

In [43]:
reconstructed_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[get_f1]
)

EPOCHS = 8
BATCH_SIZE = 128
history = reconstructed_model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // BATCH_SIZE // 2,
                    epochs=EPOCHS,
                    validation_data=valid_generator,
                    validation_steps= valid_generator.samples // BATCH_SIZE // 2,
                    verbose=1
                    )

print("#" * 100)
reconstructed_model.evaluate(valid_generator)

Epoch 1/8
10/10 [==============================] - 8s 522ms/step - loss: 0.0887 - get_f1: 0.9781 - val_loss: 0.2231 - val_get_f1: 0.9688
Epoch 2/8
10/10 [==============================] - 5s 450ms/step - loss: 0.0711 - get_f1: 0.9875 - val_loss: 0.1952 - val_get_f1: 0.9688
Epoch 3/8
10/10 [==============================] - 5s 467ms/step - loss: 0.0408 - get_f1: 0.9875 - val_loss: 0.1825 - val_get_f1: 0.9688
Epoch 4/8
10/10 [==============================] - 5s 442ms/step - loss: 0.0372 - get_f1: 0.9875 - val_loss: 0.1396 - val_get_f1: 0.9688
Epoch 5/8
10/10 [==============================] - 5s 479ms/step - loss: 0.0485 - get_f1: 0.9875 - val_loss: 0.0777 - val_get_f1: 0.9844
Epoch 6/8
10/10 [==============================] - 5s 462ms/step - loss: 0.0800 - get_f1: 0.9844 - val_loss: 0.4010 - val_get_f1: 0.9062
Epoch 7/8
10/10 [==============================] - 5s 464ms/step - loss: 0.0818 - get_f1: 0.9781 - val_loss: 0.3913 - val_get_f1: 0.9531
Epoch 8/8
21/21 [========================

[0.0863087847828865, 0.9851190447807312]